In [1]:
games = []
with open('filtered_games.pgn', 'r') as f:
    lines = []
    for line in f:
        if line == "\n":
            continue
        lines.append(line.strip())
        if len(lines) == 16:
            games.append(lines)
            lines = []


In [2]:
def parse_fen(fen_str):
    board, turn, castling, enp, halfmove, fullmove = fen_str.split(" ")
    for i in range(1, 9):
        board = board.replace(str(i), '.' * i)
    return board.replace("/", ""), turn, castling, enp, halfmove, fullmove

In [3]:
import chess.pgn
from io import StringIO

game_pgn = StringIO("\n".join(games[0]))

game = chess.pgn.read_game(game_pgn)

In [4]:
parse_fen(game.end().board().fen())

('..K.....q.......P.........k.....................................',
 'w',
 '-',
 '-',
 '4',
 '57')

In [5]:
import numpy as np

def board_to_nums(board_str):
    pieces = ['.', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']
    board_array = np.zeros(64)
    for i, element in enumerate(board_str):
        board_array[i] = pieces.index(element)
    return board_array

def castling_to_nums(castling_str):
    cnums = np.zeros(4)
    if castling_str == "-":
        return cnums
    else:
        if "K" in castling_str:
            cnums[0] = 1
        if "Q" in castling_str:
            cnums[1] = 1
        if "k" in castling_str:
            cnums[2] = 1
        if "q" in castling_str:
            cnums[3] = 1
    return cnums

In [6]:
def create_row(game_state):
    try:
        comment = re.match("\[%eval (.*)\]", game_state.comment).groups()[0]
    except:
        return None
    board_fen = game_state.board().fen()
    board, turn, castling, _, halfmove, fullmove = parse_fen(board_fen)
    return fullmove, board_to_nums(board), comment


In [7]:
import re
re.match("\[%eval (.*)\]", game.end().comment).groups()[0]

'#-2'

In [14]:
def create_dataset(game):
    data = []
    state = game.end()
    while True:
        row = create_row(state)
        if row is not None:
            fullmove, board, comment = row
            if comment[0] != "#":
                data.append(np.append(board, [comment]))
            if comment[0] == "#":
                if comment[1] == "-":
                    data.append(np.append(board, [-(50 + int(comment[2:])) * 100]))
                else:
                    data.append(np.append(board, [(50 - int(comment[1:])) * 100]))
            if int(fullmove) == 1:
                break
        try:
            state = state.parent
        except:
            break
    return np.asarray(data)

In [15]:
dataset = []
from tqdm import tqdm

for game in tqdm(games):
    game_pgn = StringIO("\n".join(game))
    try:
        game = chess.pgn.read_game(game_pgn)
        data = create_dataset(game)
        if len(data.shape) == 2:
            dataset.append(data)
    except:
        continue

  1%|          | 11/2156 [00:01<06:24,  5.58it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'e4' in rn2k2r/p4ppp/2b2b1n/8/1QP1p3/B1N1p3/P2q1PPP/R3KB1R w KQkq - 0 14
  1%|          | 20/2156 [00:02<03:24, 10.42it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], t

ValueError: illegal san: 'e4' in 3Q2k1/5p1p/4p1pP/3pPq2/8/8/6PK/8 b - - 0 43
 21%|██        | 449/2156 [00:37<01:43, 16.45it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'a6' in 8/8/2p5/4k3/2pp4/6KP/8/8 w - - 0 66
 21%|██▏       | 462/2156 [00:38<01:45, 16.02it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_ga

ValueError: illegal san: 'b7b' in 2k2Q2/1rpq2pp/8/8/1n1Pp1n1/4P1P1/5PP1/B1R3K1 b - - 10 32
 24%|██▍       | 523/2156 [00:45<03:07,  8.69it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'eb6' in 2r3r1/1p1k3p/p2Q4/2Pn1b2/1PBp4/P7/5P1P/2B1R2K b - - 1 31
 25%|██▍       | 531/2156 [00:46<02:59,  9.04it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\

ValueError: illegal san: 'c2' in 2k2r2/ppp5/5N2/r3K3/7p/7R/8/8 w - - 2 35
 30%|███       | 656/2156 [01:05<03:57,  6.32it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'f8' in 5k1r/2pn1pp1/1p1p4/3Pp2p/1PP1P1q1/3B1R2/Q5P1/2R3K1 b - - 0 26
 31%|███       | 673/2156 [01:07<04:12,  5.87it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py"

ValueError: illegal san: 'gh8' in r6r/ppkb1p1p/2n3p1/8/2B5/2Q2N2/P2N1PPP/R3R1K1 b - - 0 17
 37%|███▋      | 805/2156 [01:21<01:40, 13.39it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'e5' in r7/8/8/6Rp/Pk2K3/8/8/8 b - - 0 58
 38%|███▊      | 821/2156 [01:23<01:49, 12.15it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271

...

ValueError: illegal san: 'h6' in 2kr4/pp3ppp/2p5/8/8/3B1P2/PPP2P1P/R1K1r3 w - - 3 20
error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: '-f1' in 4k1Q1/ppr1np2/1qb5/8/8/P5P1/2P2P1P/K2RR3 b - - 0 28
error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\th

ValueError: illegal san: 'e2' in 6k1/p4p2/5R1p/2pB4/8/4r1P1/Pr5P/5RK1 b - - 2 27
 92%|█████████▏| 1983/2156 [03:08<00:11, 14.68it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'Be8' in 7Q/8/4R1bk/7p/5K1P/2P5/8/8 b - - 4 67
error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stac

ValueError: illegal san: 'b2' in b2r2k1/5p2/4p1p1/p1p1P1q1/Pp6/1P1BRPp1/2P2Q1K/8 w - - 0 38
 94%|█████████▎| 2019/2156 [03:11<00:10, 13.29it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'g1' in 5rk1/p4p1p/4nPp1/q2pP3/8/2P5/Pr1N1QPP/2R2R1K w - - 6 27
 94%|█████████▍| 2022/2156 [03:11<00:08, 15.73it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages

ValueError: illegal san: 'e4' in 8/7p/2pk4/3r1p2/4p2P/p5P1/qPQ2P2/6K1 w - - 0 35
 96%|█████████▋| 2077/2156 [03:16<00:05, 14.06it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: '-f1' in 8/1p1N2k1/2pP1b2/2P2Rp1/1P2P1Pp/8/r5K1/8 w - - 15 62
 96%|█████████▋| 2080/2156 [03:16<00:05, 13.27it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py

ValueError: illegal san: 'b4' in 8/p4pk1/1p6/2p4p/2R5/P5P1/1P3QK1/7R b - - 0 45
 99%|█████████▊| 2125/2156 [03:21<00:03,  9.28it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'ad3' in 7k/8/1p3K2/1P6/4B3/8/7P/8 b - - 2 48
 99%|█████████▉| 2133/2156 [03:21<00:02,  9.11it/s]error during pgn parsing
Traceback (most recent call last):
  File "c:\users\thepo\appdata\local\programs\python\python37\lib\site-packages\chess\pgn.py", line 1271, in 

In [16]:
len(dataset)

2155

(2,)

In [17]:
dataset = np.concatenate(dataset)

In [18]:
dataset.shape

(164401, 65)

In [19]:
np.savetxt("preprocessed.csv", dataset, delimiter=",", fmt="%s")